In [5]:
import time
import typing as t

from openpyxl import load_workbook
from datetime import datetime
from sqlalchemy.sql import func
from sqlalchemy.orm import Session

from app.core.config import settings
from app.db.session import SessionLocal

from app import crud, models, schemas

db = SessionLocal()

In [6]:
def get_file(file="reklaliste.xlsx") -> load_workbook:
    wb_file = load_workbook(str(settings.SERVER_BASE_DIR) + '/' + file)
    i = 0
    for sheet in wb_file.sheetnames:
        i += 1
        if sheet.lower() == "liste wertreklas":
            break
        wb_file.active = i
    return wb_file.active

In [7]:
def fetch_claim_data(row: t.List) -> t.List:
    data = []
    for cell in row:
        data.append(cell)
    create_claim(data=data)
    return data

In [8]:
def create_claim(*, data: t.List) -> t.Union[schemas.claim.ClaimBase, None]:
    claim_in = schemas.claim.ClaimBase()

    if isinstance(data[1], float):
        contract_nr = str(data[1]).split(".0")[0]
        claim_in.contract_nr = int(contract_nr)
    if isinstance(data[2], float) or isinstance(data[2], int):
        claim_in.discharge = float(data[2])
    if isinstance(data[3], float) or isinstance(data[3], int):
        claim_in.bill = float(data[3])
    if isinstance(data[7], datetime):
        claim_in.created_at = datetime.date(data[7])
    if isinstance(data[11], str):
        claim_in.kind = data[11].upper()
        print(claim_in.kind)
        try:
            claim_in.kind = models.ClaimKind(claim_in.kind).name
        except ValueError:
            claim_in.kind = None

    if claim_in.contract_nr is not None and claim_in.bill is not None and claim_in.created_at is not None:
        return claim_in
    return None

In [22]:
def iter_claim_rows(file: load_workbook) -> t.List[schemas.Claim]:
    i = 0
    for row in file.iter_rows(values_only=True, max_col=15):
        i += 1
        if i  == 10:
            break
        echo = fetch_claim_data(row)
        if echo is not None:
            yield echo

In [23]:
claim_file = get_file()

/usr/local/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [24]:
rows = iter_claim_rows(file=claim_file)

In [25]:
for row in rows:
    print(row)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
SCHULD LT. MELDUNG
[None, 'Auftrag Nr', 'SPEDI BA', 'INV Rekla', 'VKP Rekla', 'Rekla POS', 'TK', 'Rekla Datum', 'Lnr', 'Lieferant', 'HWG', 'Schuld lt. Meldung', 'Maßnahmengruppe', 'Verkaeufer Nr', 'Verkäufername']
LIEFERANT
[None, 1023309.0, None, 69.97, 0, 31, 2, datetime.datetime(2020, 1, 2, 0, 0), 287, 'Impuls', '8.4.3', 'Lieferant', 'Ersatz(teil)kosten', 1430, 'Frau Naumann']
MONTEUR
[None, 1068349.0, -120.8, 97.62, 0, 98, 10, datetime.datetime(2020, 1, 2, 0, 0), 400, 'artego', '8.4.3', 'Monteur', 'Ersatz(teil)kosten', 9084, 'Herr Hettinger']
MONTEUR
[None, 1099803.0, -1142.74, 93.78, 0, 116, 13, datetime.datetime(2020, 1, 2, 0, 0), 2349, 'Schock', '8.4.3', 'Monteur', 'Ersatz(teil)kosten', 2622, 'Herr Yilmaz']
VERKÄUFER
[None, 1174687.0, None, 342.36, 0, 103, 2, datetime.datetime(2020, 1, 2, 0, 0), 2643, 'Express', '8.4.3', 'Verkäufer', 'Ersatz(teil)kosten', 2572, 'Herr Schindler']
VERKÄUFER
